<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/preprocessing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download datasets
! wget 'https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/event2semantic_vec.json'
! wget 'https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/hdfs_train'
! wget 'https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/hdfs_test_normal'
! wget 'https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/hdfs_test_abnormal'

--2023-07-11 15:43:52--  https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/event2semantic_vec.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 259484 (253K) [text/plain]
Saving to: ‘event2semantic_vec.json’

event2semantic_vec. 100%[===================>] 253.40K  --.-KB/s    in 0.03s   

2023-07-11 15:43:52 (7.85 MB/s) - ‘event2semantic_vec.json’ saved [259484/259484]

--2023-07-11 15:43:52--  https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/hdfs_train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response.

In [2]:
import json
from collections import Counter

import numpy as np
import pandas as pd
from tqdm import tqdm


In [4]:
with open('/content/event2semantic_vec.json', 'r') as load_f:
  event2semantic_vec  = json.load(load_f) #it is a dictionary, for exaple, first template is '0'and each templates is 300 demension

In [6]:
len(event2semantic_vec)

29

In [11]:
len(event2semantic_vec['0'])

300

In [ ]:
num_sessions = 0
result_logs = {}
result_logs['Sequentials'] = []
result_logs['Quantitatives'] = []
result_logs['Semantics'] = []
labels = []

with open('/content/hdfs_train', 'r') as f:
    for line in f:
        num_sessions += 1
        line = [int(i) - 1 for i in line.strip().split()] # we substract by one from templates index for starting from zero
